In [8]:
#%run -i IOD-model.ipynb
%run -i common.ipynb

from lime import lime_tabular
from numpy import loadtxt
import dill
import joblib
import shap
import pickle
import pandas as pd
import scipy as sp

#data_to_train="/project/projectdirs/m2621/dbin/Cori_archive_2019_un_taz_parsered_tagged.csv"

#explain_model_save_file="/global/cfs/cdirs/m2621/dbin/io-explain-model-lime.joblib"
#explain_model_save_file_shap="/global/cfs/cdirs/m2621/dbin/io-explain-model-shap.joblib"
#predict_model_save_file="/global/cfs/cdirs/m2621/dbin/io-ai-model.joblib"

#v1
#data_to_train="/global/cfs/projectdirs/m2621/dbin/Cori_archive_19_20_21_22_un_taz_parsered_tagged_for_training.csv"
data_to_train="/global/cfs/projectdirs/m2621/dbin/Cori_archive_19_20_21_22_un_taz_parsered_tagged_for_training-v1.csv"

#predict_model_save_file_xgboost="/global/cfs/cdirs/m2621/dbin/io-ai-model-xgboost-20221019-162202.joblib"



# This is the file for Docker version now
##predict_model_save_file_xgboost="/global/cfs/cdirs/m2621/dbin/io-ai-model-xgboost-sparse-20221107-151140.joblib"
## v0 predict_model_save_file_xgboost="/global/cfs/cdirs/m2621/dbin/io-ai-model-xgb-sparse-20221214-111321.joblib"

predict_model_save_file_xgboost="/global/homes/d/dbin/IODiagnoser/web/model/io-ai-model-xgb-sparse-20230118-213200-v1.joblib"
explain_model_save_file_shap_xgboost="/global/homes/d/dbin/IODiagnoser/web/model/io-explain-model-shap-xgboost-v1.joblib"
explain_model_save_file_lime_xgboost="/global/homes/d/dbin/IODiagnoser/web/model/io-explain-model-lime-xgboost-v1.joblib"


# This is the file for Docker version now
#predict_model_save_file_tabnet="/global/cfs/cdirs/m2621/dbin/io-ai-model-tabnet-cpu-20221102-132100.joblib"
#predict_model_save_file_tabnet="/global/cfs/cdirs/m2621/dbin/io-ai-model-tabnet-20221216-231906.joblib"
#explain_model_save_file_shap_tabnet ="/global/cfs/cdirs/m2621/dbin/io-explain-model-shap-tabnet.joblib"
#explain_model_save_file_lime_tabnet ="/global/cfs/cdirs/m2621/dbin/io-explain-model-lime-tabnet.joblib"

predict_model_save_file_tabnet="/global/homes/d/dbin/IODiagnoser/web/model/io-ai-model-tabnet-20230119-061549-v1.joblib"
explain_model_save_file_lime_tabnet ="/global/homes/d/dbin/IODiagnoser/web/model/io-explain-model-lime-tabnet-v1.joblib"


# This is the file for Docker version now
#predict_model_save_file_mlp="/global/cfs/cdirs/m2621/dbin/io-ai-model-mlp-sparse-20221108-110917.joblib"
#v0
#predict_model_save_file_mlp="/global/cfs/cdirs/m2621/dbin/io-ai-model-mlp-sparse-20221211-094606.joblib"
#v1
predict_model_save_file_mlp         ="/global/homes/d/dbin/IODiagnoser/web/model/io-ai-model-mlp-sparse-20230118-212203-v1.joblib"
explain_model_save_file_shap_mlp    ="/global/homes/d/dbin/IODiagnoser/web/model/io-explain-model-shap-mlp-v1.joblib"
explain_model_save_file_lime_mlp    ="/global/homes/d/dbin/IODiagnoser/web/model/io-explain-model-lime-mlp-v1.joblib"


#v0
#predict_model_save_file_lightgbm="/global/cfs/cdirs/m2621/dbin/io-ai-model-lightGBM-sparse-20230102-173739.joblib"
#explain_model_save_file_shap_lightgbm="/global/cfs/cdirs/m2621/dbin/io-explain-model-shap-lightgbm.joblib"
#explain_model_save_file_lime_lightgbm="/global/cfs/cdirs/m2621/dbin/io-explain-model-lime-lightgbm.joblib"

predict_model_save_file_lightgbm="/global/homes/d/dbin/IODiagnoser/web/model/io-ai-model-lightgbm-sparse-20230119-120849-v1.joblib"
explain_model_save_file_shap_lightgbm="/global/homes/d/dbin/IODiagnoser/web/model/io-explain-model-shap-lightgbm-v1.joblib"
explain_model_save_file_lime_lightgbm="/global/homes/d/dbin/IODiagnoser/web/model/io-explain-model-lime-lightgbm-v1.joblib"


#predict_model_save_file_catboost="/global/cfs/cdirs/m2621/dbin/io-ai-model-catboost-sparse-20230105-214802.joblib"
predict_model_save_file_catboost     ="/global/homes/d/dbin/IODiagnoser/web/model/io-ai-model-catboost-sparse-20230119-120849-v1.joblib"
explain_model_save_file_shap_catboost="/global/homes/d/dbin/IODiagnoser/web/model/io-explain-model-shap-catboost-v1.joblib"
explain_model_save_file_lime_catboost="/global/homes/d/dbin/IODiagnoser/web/model/io-explain-model-lime-catboost-v1.joblib"


def load_train_data_to_explain_model(file_csv_to_train, method="LIME"):
    if method == "LIME":
        ## Get the headers from the explain dataset
        f = open(file_csv_to_train)
        header = f.readline()
        dataset_headers = header.split(',')
        dataset_headers.pop()
        f.close()
        ## Get the data used to train the model
        train_dataset = loadtxt(file_csv_to_train, delimiter=',', skiprows=1) ## skiprows=1  to skip header
        n_dims = train_dataset.shape[1]
        train_X = train_dataset[:,0:n_dims-1]
        ##train_Y = train_dataset[:,n_dims-1]
        return train_X, dataset_headers
    elif method == "SHAP":
        df=pd.read_csv(file_csv_to_train)
        #train_Y=df["tag"].to_numpy()
        df.drop(['tag'], axis=1, inplace=True)
        dataset_headers=list(df.columns)
        return df, dataset_headers
    else:
        print("Not supported explain method now !\n")
        exit()
        
def create_save_explain_model_LIME(file_to_save, is_train_x_dense=False):
    ## Build the LimeTabularExplainer
    train_X, dataset_headers=load_train_data_to_explain_model(data_to_train, method="LIME")
    print(train_X.shape)
    print(dataset_headers)
    if is_train_x_dense:
        train_X_to_go = train_X
    else:
        train_X_to_go = sp.sparse.csr_matrix(train_X)

    explainer = lime_tabular.LimeTabularExplainer(train_X_to_go, mode="regression", feature_names= dataset_headers)
    with open(file_to_save, 'wb') as f:
        dill.dump(explainer, f)
    #with open('data', 'rb') as f:
    #dill.load(f)

    
## Good example of shap
## https://github.com/slundberg/shap/issues/201
## https://towardsdatascience.com/explainable-ai-xai-with-shap-regression-problem-b2d63fdca670
## https://www.yourdatateacher.com/2021/05/17/how-to-explain-neural-networks-using-shap/
def create_save_explain_model_SHAP(file_to_save, predict_model_file_p, is_train_x_dense=False):
    train_X, dataset_headers=load_train_data_to_explain_model(data_to_train, method="LIME")
    print(train_X.shape)
    print(dataset_headers)
    ## Since we already have CPU version of Tabnet, we only need load it as normal joblib file
    predict_model = load_predict_model(predict_model_file_p, isTabnet = False); 
    if is_train_x_dense == True:
        explainer = shap.Explainer(predict_model.predict) 
    else:
        # https://github.com/slundberg/shap/blob/master/tests/explainers/test_kernel.py#L76
        # follow example of test_kernel_shap_with_a1a_sparse_zero_background():
        _, cols = train_X.shape
        shape = 1, cols
        print("shape =", shape)
        background = sp.sparse.csr_matrix(shape, dtype=train_X.dtype)
        #print(background.toarray())
        explainer = shap.KernelExplainer(predict_model.predict, background) 
    
    joblib.dump(explainer, filename=file_to_save)
    
    #train_X_summary=shap.sample(train_X, 3000)
    #masker = shap.maskers.Independent(data = train_X_summary)
    #shap_values2 = explainer(train_X.head(10))
    #shap.plots.waterfall(shap_values2[0])
    ##shap.plots.waterfall(shap_values2[0])
    #train_X_summary = shap.kmeans(train_X, 300000)
    #train_X_summary=shap.sample(train_X, 3000)
    #masker = shap.maskers.Independent(data = train_X_summary)
    #explainer = shap.KernelExplainer(predict_model.predict, train_X_summary, masker = masker) 
    #explainer = shap.SamplingExplainer(predict_model.predict, train_X)
    #explainer = shap.PermutationExplainer(predict_model.predict)
    #explainer = shap.DeepExplainer(predict_model.predict, train_X_summary) 
    #joblib.dump(explainer, filename=file_to_save)
    ##explainer = shap.Explainer(predict_model.predict, train_X)
    ##shap_values2 = explainer.shap_values(train_X.head(2))
    ##shap.plots.waterfall(shap_values2[0])
    ##explain_model=joblib.load(filename=file_to_save)
    ##shap_values = explain_model(train_X.head(2))
    ##shap.plots.waterfall(shap_values[0])



def create_save_explain_model(file_to_save_p, explain_method = "LIME", predict_model_file=None, is_train_x_dense_p=False):
    if explain_method == "LIME":
        create_save_explain_model_LIME(file_to_save=file_to_save_p, is_train_x_dense=is_train_x_dense_p);
    elif explain_method == "SHAP":
        create_save_explain_model_SHAP(file_to_save=file_to_save_p, predict_model_file_p=predict_model_file, is_train_x_dense=is_train_x_dense_p)
    else:
        print("Not supported explain method now !\n")
        exit()

#create_save_explain_model(explain_method = "SHAP", file_to_save_p=explain_model_save_file_shap_mlp,     predict_model_file=predict_model_save_file_mlp)
#create_save_explain_model(explain_method = "LIME", file_to_save_p=explain_model_save_file_lime_mlp,     predict_model_file=predict_model_save_file_mlp)

#create_save_explain_model(explain_method = "LIME", file_to_save_p=explain_model_save_file_lime_xgboost, predict_model_file=predict_model_save_file_xgboost)
#create_save_explain_model(explain_method = "SHAP", file_to_save_p=explain_model_save_file_shap_xgboost, predict_model_file=predict_model_save_file_xgboost)

        
## This is no need to train SHAP with Tabnet, it is created in IOD-app druing running
##create_save_explain_model(explain_method = "SHAP", file_to_save_p=explain_model_save_file_shap_tabnet,  predict_model_file=predict_model_save_file_tabnet, is_train_x_dense_p=True)
## comment out above line      
create_save_explain_model(explain_method = "LIME", file_to_save_p=explain_model_save_file_lime_tabnet,  predict_model_file=predict_model_save_file_tabnet)


#create_save_explain_model(explain_method = "SHAP", file_to_save_p=explain_model_save_file_shap_lightgbm, predict_model_file=predict_model_save_file_lightgbm)
#create_save_explain_model(explain_method = "LIME", file_to_save_p=explain_model_save_file_lime_lightgbm, predict_model_file=predict_model_save_file_lightgbm)


#create_save_explain_model(explain_method = "SHAP", file_to_save_p=explain_model_save_file_shap_catboost, predict_model_file=predict_model_save_file_catboost)
#create_save_explain_model(explain_method = "LIME", file_to_save_p=explain_model_save_file_lime_catboost, predict_model_file=predict_model_save_file_catboost)


print("Done")


(6647219, 45)
['nprocs', 'POSIX_OPENS', 'LUSTRE_STRIPE_SIZE', 'LUSTRE_STRIPE_WIDTH', 'POSIX_FILENOS', 'POSIX_MEM_ALIGNMENT', 'POSIX_FILE_ALIGNMENT', 'POSIX_READS', 'POSIX_WRITES', 'POSIX_SEEKS', 'POSIX_STATS', 'POSIX_BYTES_READ', 'POSIX_BYTES_WRITTEN', 'POSIX_CONSEC_READS', 'POSIX_CONSEC_WRITES', 'POSIX_SEQ_READS', 'POSIX_SEQ_WRITES', 'POSIX_RW_SWITCHES', 'POSIX_MEM_NOT_ALIGNED', 'POSIX_FILE_NOT_ALIGNED', 'POSIX_SIZE_READ_0_100', 'POSIX_SIZE_READ_100_1K', 'POSIX_SIZE_READ_1K_10K', 'POSIX_SIZE_READ_100K_1M', 'POSIX_SIZE_WRITE_0_100', 'POSIX_SIZE_WRITE_100_1K', 'POSIX_SIZE_WRITE_1K_10K', 'POSIX_SIZE_WRITE_10K_100K', 'POSIX_SIZE_WRITE_100K_1M', 'POSIX_STRIDE1_STRIDE', 'POSIX_STRIDE2_STRIDE', 'POSIX_STRIDE3_STRIDE', 'POSIX_STRIDE4_STRIDE', 'POSIX_STRIDE1_COUNT', 'POSIX_STRIDE2_COUNT', 'POSIX_STRIDE3_COUNT', 'POSIX_STRIDE4_COUNT', 'POSIX_ACCESS1_ACCESS', 'POSIX_ACCESS2_ACCESS', 'POSIX_ACCESS3_ACCESS', 'POSIX_ACCESS4_ACCESS', 'POSIX_ACCESS1_COUNT', 'POSIX_ACCESS2_COUNT', 'POSIX_ACCESS3_COUNT